# Import Modules
This section imports the required modules

In [1]:
import pandas as pd
from zipfile import ZipFile

In [11]:
zip_path = './data/human-trafficking-victims-dataset-ctdc.zip'
data_file = None
raw_data = None
with ZipFile(zip_path) as zip_file:
    for commpresed in zip_file.filelist:
        if commpresed.filename.endswith('csv'):
            with zip_file.open(commpresed) as data_file:
                raw_data = pd.read_csv(
                    data_file
                )

In [ ]:

print("hola mundo ")